In [ ]:
import sys
sys.path.append("../")
import os
import pandas as pd
import numpy as np
import glob
import subprocess
import os

In [ ]:
best_model = 'BEST_MODEL_EVER' #model you want to use for getting predictions
dataset_path = 'ORDERED_TESTSET' # testset that is NOT shuffled
targets_i = 0 #which target or cell line to write as bw
out_dir = utils.make_dir('OUTPUT_DIR')
chrom_size_path='GENOME_SIZE_PATH'
model, bin_size = tfr_evaluate.read_model(best_model)

In [ ]:
testset, targets = tfr_evaluate.collect_whole_testset(data_dir=dataset_path, 
                                                      coords=True) # load testset
C, X, Y = utils.convert_tfr_to_np(testset, 3) # convert to np
preds = embed.predict_np(X, model, batch_size=32, reshape_to_2D=False) # get predictions

In [ ]:
# create paths for new bs files
pred_bw_filename = os.path.join(out_dir, 'pred_{}.bw'.format(targets[targets_i]))
truth_bw_filename = os.path.join(out_dir, 'truth_{}.bw'.format(targets[targets_i]))
# open 2 bws
pred_bw = test_to_bw_fast.open_bw(pred_bw_filename, chrom_size_path)
truth_bw = test_to_bw_fast.open_bw(truth_bw_filename, chrom_size_path)

In [ ]:
clean_C = [str(c).strip('b\'').strip('\'') for c in C]
for i in range(preds.shape[0]): # per data point 
    chrom, start, end = clean_C[i].split('_') # get chr, start, end
    start = int(start) # to feed into bw making function
    # for cell_line in [8]: # per cell line
    for t in [targets_i]: # per cell line
        # write to ground truth file
        truth_bw.addEntries(chrom, start,
            values=np.array(np.squeeze(Y[i,:,t]), dtype='float64'),
            span=1, step=1)
        # write to prediction bw file
        pred_bw.addEntries(chrom, start,
            values=np.array(np.squeeze(preds[i,:,t]), dtype='float64'),
            span=bin_size, step=bin_size)

In [ ]:
truth_bw.close()
pred_bw.close()